In [1]:
from models import TopDownInferenceModel, TopDownIDInferenceModel, SingleInstanceInferenceModel
import tensorflow as tf
from video import read_frames
import numpy as np
import system
from peak_finding import describe_tensors
from trtutils import convert_to_trt, OptimizedModel

system.disable_preallocation()
system.summary()

GPUs: 1/1 available
  Device: /physical_device:GPU:0
         Available: True
        Initalized: False
     Memory growth: True


In [2]:
mice_of_centroid = "sleap-data/datasets/wang_4mice_john/labels.full/models.random_split1/centroid.210507_132508.centroid.n=900"
mice_of_td = "sleap-data/experiments/best_model_replicates/mice_of_td.210509_231024.centered_instance.n=900"
mice_of_clips = """
sleap-data/datasets/wang_4mice_john/clips/OFTephys-0055-08@14616-18736.mp4
sleap-data/datasets/wang_4mice_john/clips/OFTsocialgroup-0000-00@117000-121700.mp4
sleap-data/datasets/wang_4mice_john/clips/OFTsocial5mice-0000-00.mp4.old.predictions@3700-6260.mp4
""".strip().split()


flies13_centroid = "sleap-data/datasets/wt_gold.13pt/models.tracking_split2/centroid.fast.210504_182918.centroid.n=1800"
flies13_td = "sleap-data/datasets/wt_gold.13pt/models.tracking_split2/td_fast.210505_012601.centered_instance.n=1800"
flies13_td_id = "sleap-data/datasets/wt_gold.13pt/models.tracking_split2/td_id.fast.v2.210519_111253.multi_class_topdown.n=1800"
flies13_clips = """
sleap-data/datasets/wt_gold.13pt/clips/single_fly@100000-110000.mp4
sleap-data/datasets/wt_gold.13pt/clips/190719_090330_wt_18159206_rig1.2@15000-17560.mp4
sleap-data/datasets/wt_gold.13pt/clips/three_flies@8800-11000.mp4
sleap-data/datasets/wt_gold.13pt/clips/four_flies@1000-11000.mp4
sleap-data/datasets/wt_gold.13pt/clips/eight_flies@180000-200000.mp4
""".strip().split()


fly32_single = "sleap-data/datasets/BermanFlies/models.random_split1/single.fast_unet32.210524_171130.single_instance.n=1350"
fly32_clips = ["sleap-data/datasets/BermanFlies/clips/072212_163153@10000-13200.mp4"]


gerbils_centroid = "sleap-data/datasets/nyu-gerbils/cohort1_compressedTalmo_23vids_march_7_to_march_17/models.random_split1.day001/centroid.210504_225945.centroid.n=383"
gerbils_td_id = "sleap-data/datasets/nyu-gerbils/cohort1_compressedTalmo_23vids_march_7_to_march_17/models.random_split1.day001/td_id.210505_002058.multi_class_topdown.n=383"
gerbils_clips = ["sleap-data/datasets/nyu-gerbils/clips/2020-3-10_daytime_5mins_compressedTalmo@3200-5760.mp4"]

In [3]:
centroid_model_path = mice_of_centroid
td_model_path = mice_of_td
clip_path = mice_of_clips[2]

centroid_model = tf.keras.models.load_model(f"{centroid_model_path}/best_model.h5", compile=False)
td_model = tf.keras.models.load_model(f"{td_model_path}/best_model.h5", compile=False)

inference_model = TopDownInferenceModel(
    centroid_base_model=centroid_model,
    centroid_input_size=tuple(centroid_model.inputs[0].shape[1:3]),
    centroid_input_scale=0.5,
    centroid_output_stride=centroid_model.inputs[0].shape[1] // centroid_model.outputs[0].shape[1],
    crop_size=td_model.inputs[0].shape[1],
    td_base_model=td_model,
    td_output_stride=td_model.inputs[0].shape[1] // td_model.outputs[0].shape[1],
)
test_imgs = read_frames(clip_path, np.arange(16))
print(test_imgs.shape)
print(centroid_model.inputs, centroid_model.outputs)
print(td_model.inputs, td_model.outputs)

preds = inference_model.predict(test_imgs)
describe_tensors(preds)

inference_model.save(f"{td_model_path}/savedmodel", save_format="tf", save_traces=True)
convert_to_trt(f"{td_model_path}/savedmodel", f"{td_model_path}/trtmodel", precision="FP16")
convert_to_trt(f"{td_model_path}/savedmodel", f"{td_model_path}/trtmodel", precision="FP32")

# trt_model = OptimizedModel(f"{td_model_path}/trtmodel_FP16")
# preds = trt_model.predict(test_imgs)
# describe_tensors(preds)

(16, 1280, 1024, 1)
[<KerasTensor: shape=(None, 640, 512, 1) dtype=float32 (created by layer 'input')>] [<KerasTensor: shape=(None, 320, 256, 1) dtype=float32 (created by layer 'CentroidConfmapsHead_0')>]
[<KerasTensor: shape=(None, 352, 352, 1) dtype=float32 (created by layer 'input')>] [<KerasTensor: shape=(None, 88, 88, 11) dtype=float32 (created by layer 'CenteredInstanceConfmapsHead_0')>]
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
    instance_peaks: type=ndarray, shape=(80, 11, 2), dtype=float32, device=N/A
instance_peak_vals: type=ndarray, shape=(80, 11), dtype=float32, device=N/A
       sample_inds: type=ndarray, shape=(80,), dtype=int32, device=N/A
INFO:tensorflow:Assets written to: sleap-data/experiments/best_model_replicates/mice_of_td.210509_231024.centered_instance.n=900/savedmodel/assets
Created ModelOptimizer with: sleap-data/experiments/best_model_replicates/mice_of_td.210509_

True

In [4]:
model_path = fly32_single
clip_path = fly32_clips[0]

model = tf.keras.models.load_model(f"{model_path}/best_model.h5", compile=False)

inference_model = SingleInstanceInferenceModel(
    base_model=model,
    output_stride=model.inputs[0].shape[1] // model.outputs[0].shape[1],
)
test_imgs = read_frames(clip_path, np.arange(4))
preds = inference_model.predict(test_imgs)



inference_model.save(f"{model_path}/savedmodel", save_format="tf", save_traces=True)
convert_to_trt(f"{model_path}/savedmodel", f"{model_path}/trtmodel", precision="FP16")
convert_to_trt(f"{model_path}/savedmodel", f"{model_path}/trtmodel", precision="FP32")

INFO:tensorflow:Assets written to: sleap-data/datasets/BermanFlies/models.random_split1/single.fast_unet32.210524_171130.single_instance.n=1350/savedmodel/assets
Created ModelOptimizer with: sleap-data/datasets/BermanFlies/models.random_split1/single.fast_unet32.210524_171130.single_instance.n=1350/savedmodel
INFO:tensorflow:Linked TensorRT version: (7, 2, 2)
INFO:tensorflow:Loaded TensorRT version: (7, 2, 3)
INFO:tensorflow:Loaded TensorRT 7.2.3 and linked TensorFlow against TensorRT 7.2.2. This is supported because TensorRT  minor/patch upgrades are backward compatible
INFO:tensorflow:Could not find TRTEngineOp_2_6 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_2_4 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_2_5 in TF-TRT cache. This can happe

True

In [5]:
# td_model_path = mice_of_td
# clip_path = mice_of_clips[1]
# trt_model = OptimizedModel(f"{td_model_path}/trtmodel_FP16")


# test_imgs = read_frames(clip_path, np.arange(4))
# trt_model.predict(test_imgs)

In [6]:
centroid_model_path = flies13_centroid
td_model_path = flies13_td

centroid_model = tf.keras.models.load_model(f"{centroid_model_path}/best_model.h5", compile=False)
td_model = tf.keras.models.load_model(f"{td_model_path}/best_model.h5", compile=False)

inference_model = TopDownInferenceModel(
    centroid_base_model=centroid_model,
    centroid_input_size=tuple(centroid_model.inputs[0].shape[1:3]),
    centroid_input_scale=0.5,
    centroid_output_stride=centroid_model.inputs[0].shape[1] // centroid_model.outputs[0].shape[1],
    crop_size=td_model.inputs[0].shape[1],
    td_base_model=td_model,
    td_output_stride=td_model.inputs[0].shape[1] // td_model.outputs[0].shape[1],
)
test_imgs = read_frames(flies13_clips[1], np.arange(4))
preds = inference_model.predict(test_imgs)
inference_model.save(f"{td_model_path}/savedmodel", save_format="tf", save_traces=True)
convert_to_trt(f"{td_model_path}/savedmodel", f"{td_model_path}/trtmodel", precision="FP16")
convert_to_trt(f"{td_model_path}/savedmodel", f"{td_model_path}/trtmodel", precision="FP32")

INFO:tensorflow:Assets written to: sleap-data/datasets/wt_gold.13pt/models.tracking_split2/td_fast.210505_012601.centered_instance.n=1800/savedmodel/assets
Created ModelOptimizer with: sleap-data/datasets/wt_gold.13pt/models.tracking_split2/td_fast.210505_012601.centered_instance.n=1800/savedmodel
INFO:tensorflow:Linked TensorRT version: (7, 2, 2)
INFO:tensorflow:Loaded TensorRT version: (7, 2, 3)
INFO:tensorflow:Loaded TensorRT 7.2.3 and linked TensorFlow against TensorRT 7.2.2. This is supported because TensorRT  minor/patch upgrades are backward compatible
INFO:tensorflow:Could not find TRTEngineOp_4_0 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_4_10 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_4_8 in TF-TRT cache. This can happen if build(

True

In [7]:
centroid_model_path = flies13_centroid
td_model_path = flies13_td_id
clip_path = flies13_clips[1]

centroid_model = tf.keras.models.load_model(f"{centroid_model_path}/best_model.h5", compile=False)
td_model = tf.keras.models.load_model(f"{td_model_path}/best_model.h5", compile=False)

inference_model = TopDownIDInferenceModel(
    centroid_base_model=centroid_model,
    centroid_input_size=tuple(centroid_model.inputs[0].shape[1:3]),
    centroid_input_scale=0.5,
    centroid_output_stride=centroid_model.inputs[0].shape[1] // centroid_model.outputs[0].shape[1],
    crop_size=td_model.inputs[0].shape[1],
    td_base_model=td_model,
    td_output_stride=td_model.inputs[0].shape[1] // td_model.outputs[0].shape[1],
)
test_imgs = read_frames(clip_path, np.arange(4))
preds = inference_model.predict(test_imgs)
inference_model.save(f"{td_model_path}/savedmodel", save_format="tf", save_traces=True)
convert_to_trt(f"{td_model_path}/savedmodel", f"{td_model_path}/trtmodel", precision="FP16")
convert_to_trt(f"{td_model_path}/savedmodel", f"{td_model_path}/trtmodel", precision="FP32")

INFO:tensorflow:Assets written to: sleap-data/datasets/wt_gold.13pt/models.tracking_split2/td_id.fast.v2.210519_111253.multi_class_topdown.n=1800/savedmodel/assets
Created ModelOptimizer with: sleap-data/datasets/wt_gold.13pt/models.tracking_split2/td_id.fast.v2.210519_111253.multi_class_topdown.n=1800/savedmodel
INFO:tensorflow:Linked TensorRT version: (7, 2, 2)
INFO:tensorflow:Loaded TensorRT version: (7, 2, 3)
INFO:tensorflow:Loaded TensorRT 7.2.3 and linked TensorFlow against TensorRT 7.2.2. This is supported because TensorRT  minor/patch upgrades are backward compatible
INFO:tensorflow:Could not find TRTEngineOp_6_0 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_6_10 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_6_8 in TF-TRT cache. This can 

True

In [3]:
centroid_model_path = gerbils_centroid
td_model_path = gerbils_td_id
clip_path = gerbils_clips[0]

centroid_model = tf.keras.models.load_model(f"{centroid_model_path}/best_model.h5", compile=False)
td_model = tf.keras.models.load_model(f"{td_model_path}/best_model.h5", compile=False)

inference_model = TopDownIDInferenceModel(
    centroid_base_model=centroid_model,
    centroid_input_size=tuple(centroid_model.inputs[0].shape[1:3]),
    centroid_input_scale=0.5,
    centroid_output_stride=centroid_model.inputs[0].shape[1] // centroid_model.outputs[0].shape[1],
    crop_size=td_model.inputs[0].shape[1],
    td_base_model=td_model,
    td_output_stride=td_model.inputs[0].shape[1] // td_model.outputs[0].shape[1],
)
test_imgs = read_frames(clip_path, np.arange(4), grayscale=False)
preds = inference_model.predict(test_imgs)
inference_model.save(f"{td_model_path}/savedmodel", save_format="tf", save_traces=True)
convert_to_trt(f"{td_model_path}/savedmodel", f"{td_model_path}/trtmodel", precision="FP16")
convert_to_trt(f"{td_model_path}/savedmodel", f"{td_model_path}/trtmodel", precision="FP32")

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
INFO:tensorflow:Assets written to: sleap-data/datasets/nyu-gerbils/cohort1_compressedTalmo_23vids_march_7_to_march_17/models.random_split1.day001/td_id.210505_002058.multi_class_topdown.n=383/savedmodel/assets
Created ModelOptimizer with: sleap-data/datasets/nyu-gerbils/cohort1_compressedTalmo_23vids_march_7_to_march_17/models.random_split1.day001/td_id.210505_002058.multi_class_topdown.n=383/savedmodel
INFO:tensorflow:Linked TensorRT version: (7, 2, 2)
INFO:tensorflow:Loaded TensorRT version: (7, 2, 3)
INFO:tensorflow:Loaded TensorRT 7.2.3 and linked TensorFlow against TensorRT 7.2.2. This is supported because TensorRT  minor/patch upgrades are backward compatible
INFO:tensorflow:Could not find TRTEngineOp_0_0 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:

True